<h1>Importa datos de competencias y del foro de alumns de las copias de seguridad de las tutorias


In [ ]:
import xml.etree.ElementTree as ET
import re
import time
import glob

### Funciones útiles

In [ ]:
def packXml(pathtofile):
    fich=open(pathtofile,'r')
    text='\n'.join(fich.readlines())
    fich.close()
    return text


def extrae_xml(course):
    '''
    Devuelve dos diccionarios con información sobre cada alumno del curso
    dforum: Información en los foros
    dstudent: Información sobre competencias en las encuestas del foro
    
    course: path al directorio en el que se extrajo la copia de seguridad'''
    
    
    text=packXml(course+'/users.xml')
    tree = ET.fromstring(text)
    idents={} #diccionario con la lista de usuarios del grupo {AVId:{nome/apelido}}
    
    #Carga la lista de usuarios del curso
    for item in tree.iter('user'):
        nome=item.find('firstname').text
        apelido=item.find('lastname').text
        AVId=int(item.get('id'))
        
        idents[AVId]=idents.get(AVId,{'nome':nome,'apelido':apelido})
        
    
   
    dforum={} #diccionario para almacenar comentarios de los foros {nalum:[(identUser,date,message),...]}
    dstudent={} #diccionario para almacenar resultados de competencias
    #{nalum:{competencia:{nivel:[DocId,...]}}}
    
    for activ in glob.glob(course+'/activities/*'):    
        
        text=packXml(activ+'/inforef.xml')
        act=ET.fromstring(text)
        num_users=len(act.findall('userref/user'))
        
        #Si la actividad tiene usuarios
        if num_users:
            
            #Obtener el nombre de la actividad
            suffix=re.findall('([a-z]+)_[0-9]+',activ)[0]
            
            act=ET.fromstring(packXml(activ+'/'+suffix+'.xml'))
            act_name=act.find(suffix+'/name').text
            
            #Si es un foro y el nombre incluye la subcadena 'alumn'
            if suffix=='forum' and 'alumn' in act_name:
                for item in act.iter('discussion'):
                    
                    #Se extrae el nombre del hilo, debe ser un nombre de alumno válido 
                    nalum=item.find('name').text
                    
                    #Se extrae el usuario que hace el post, el contenido y la fecha del post
                    for quest in item.iter('post'):
                        ident=quest.find('userid').text
                        message= quest.find('message').text
                        while re.search('<',message):
                            message=re.sub('<.*?>','',message)
                        
                        message=re.sub('\n+','\n',message)
                        date=time.strftime('%d/%b/%Y',time.gmtime(int(quest.find('modified').text)))
                        dforum[nalum]=dforum.get(nalum,[])+[(ident,date,message)]
                        
            elif suffix=='feedback':
            #Si es una encuesta 
                ok=False
                for comp in COMPETENCIAS:
                    if comp in act_name:
                        ok=True
                        break
                #Si el nombre de la encuesta incluye como subcadena un de las competencias clave
                if ok:
                    
                    qid={item.get('id'):item.find('name').text for item in act.iter('item') if item}
                    for item in act.iter('completed'):

                        if item:
                            docId=int(item.find('userid').text)
                            for value in item.iter('value'):
                                if value:
                                    nstudent=qid[value.find('item').text]
                                    
                                    if nstudent in set(dstudent.keys()):
                                        dstudent[nstudent][COMPETENCIAS.index(comp)][int(value.find('value').text)-1]+=[docId]
                                    else:
                                        dstudent[nstudent]=[[[],[],[],[],[]] for dummy in range(8)]
                                        dstudent[nstudent][COMPETENCIAS.index(comp)][int(value.find('value').text)-1]+=[docId]
                                                 


    return dstudent,dforum                          
        
        
    

def procesa_Comp(dstudent,aval='1'):
    '''
    Incorpora la información extraída con extrae_xml() a la AVBD
    dstudent diccionario con la información de competencias devuelto por extrae_xml
    aval: número de la evaluación a que refire la información
    
    Usa el objeto global avbd, conexión a la AVBD actual'''

    for alum in dstudent.keys():
        apelido,nome=alum.split(',')
        RolId=avbd.buscaRolId(nome=nome.strip(),apelido=apelido.strip())
        if not RolId:
            RolId=avbd.buscaRolId(nome=apelido.strip(),apelido=nome.strip())
            if not RolId:
                print('Alumn desconoc: ',alum)
                continue
        RolId=RolId[0][0]
        for compet,vals in zip(COMPETENCIAS,dstudent[alum]):
            if 'obx' in compet:
                compet='obx'
            ListaCompId=avbd.busca('SELECT id FROM ListaComp WHERE abrev LIKE "%s"' % compet.lower())[0][0]
            for index,item in enumerate(vals[:-1]):
                for doc in item:
                    DocId=avbd.busca('SELECT RolId FROM Docente_IP WHERE AVId LIKE "%s"' % doc)[0][0]
                    cad='INSERT INTO Competencias(RolId,DocenteId,ListaCompId,data,aval,valor) VALUES (%i,%i,%i,"%s","%s",%i)' % (RolId,DocId,ListaCompId,ANNO,aval,index)
                    try:
                        avbd.ejecuta(cad)
                    except:
                        pass
    avbd.commit()
        
    


def procesa_Forum(dforum):
    '''
    Incorpora la información extraída con extrae_xml() a la AVBD
    dfoum diccionario con la información en los foros devuelto por extrae_xml
    
    Usa el objeto global avbd, conexión a la AVBD actual'''
    
    for alumk in dforum.keys():
        alum=alumk
        cand=set()
        
        #Intenta encontrar de forma unívoca un alumno usuario del AVBD que case con el nombre del hilo del foro
        if len(alum.split())>=7:
            continue
        if ',' in alum:
            apelido,nome=alum.split(',')
            RolId=avbd.buscaRolId(nome=nome.strip().lower(),apelido=apelido.strip().lower())
            
            if not RolId:
                alum=re.sub(',',' ',alum)
            else:
                RolId=RolId[0][0]
                
        if ',' not in alum or not RolId:
            
            
            for item in alum.split():
                avatar=item.strip()
                avatar=avatar[0]+avatar[1:].lower()
                RolId=avbd.buscaRolId(nome=avatar)
                if RolId: setn=set(RolId)
                else: setn=set()
                RolId=avbd.buscaRolId(apelido=avatar)
                if RolId: seta=set(RolId)
                else: seta=set()
                if len(seta)>len(setn):
                    set1=seta
                    set2=setn
                else:
                    set1=setn
                    set2=seta
                if not cand:
                    ok=True
                    cand=cand.union(set1)
                else:
                    if cand.intersection(set1):
                        cand=cand.intersection(set1)
                    else:
                        if set2 and cand.intersection(set2):
                            cand=cand.intersection(set2)
            if len(cand)==1:
                RolId=list(cand)[0][0]
            else:
                RolId=None
        #Si no se consigue una correspondencia unívoca entre usuario AVBD y nombre del foro se ignora la información
        #y se genera un aviso
        if not RolId:
            if cand:
                print(alum,'-->',cand)
            else:
                print(alum,'···>',RolId)
        if RolId:
            for DocId,data,mess in dforum[alumk]:
                cad='INSERT INTO Forum(rolId,docId,data,coment) VALUES (%i,%i,"%s","%s")' % (RolId,int(DocId),data,re.sub('"','`',mess))
                try:
                    avbd.ejecuta(cad)
                except Exception as e:
                    pass
                
                    
    avbd.commit()


### Proceso de las copias de seguridad
procesa el lote de copias de seguridad Moodle (*.mbz) situado en el directorio _PATH+SUFF_

In [ ]:
#Inicialización del sistema


from AVBD import AV_BD
avbd=AV_BD('Path-to-AVBD')

PATH='Path-to-root AVBD'
ANNO='Name-of-actual-AVBD-directory'
SUFF='CopsSeg/'

#Denominación de las competencias clave y objetivos en las encuestas
COMPETENCIAS=['CCL',
'CMCCT',
'CD',
'CAA',
'CSC',
'CSIEE',
'CCEC',
'obxectivos']


#proceso archivos mbz --> zip --> extract

import os,zipfile

for item in glob.glob(PATH+SUFF+'*.mbz'):
    nomes=re.findall(r'-(titd.{5,5})-',item)
    nomes=nomes[0]
    os.rename(item,PATH+SUFF+nomes+'.mbz')
    with zipfile.ZipFile(PATH+SUFF+nomes+'.mbz',"r") as zip_ref:
        zip_ref.extractall(PATH+SUFF+nomes)

# seleccion solo de los directorios, nombres sin punto
courses=(set(glob.glob(PATH+SUFF+'*'))-set(glob.glob(PATH+SUFF+'*.*')))

for course in courses:
    print('\nCurso: ',course)
    dstudent,dforum=extrae_xml(course)
    procesa_Forum(dforum)
    procesa_Comp(dstudent,3)
    